In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('Sample - Superstore.csv',encoding='latin-1')

In [4]:
print("Dataset loaded successfully!")
print(f"Shape: {data.shape}")
print("Columns:", data.columns.tolist()[:10])  # Show first 10 columns

Dataset loaded successfully!
Shape: (9994, 21)
Columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City']


In [5]:
# Basic EDA
print(f"\n DATASET OVERVIEW")
print(f"Shape: {data.shape}")
print(f"Columns: {len(data.columns)}")
print("\nColumn Names:")
print(data.columns.tolist())
print("\nData Types:")
print(data.dtypes)
print("\nMissing Values:")
print(data.isnull().sum())


 DATASET OVERVIEW
Shape: (9994, 21)
Columns: 21

Column Names:
['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']

Data Types:
Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code        int64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
Quantity           int64
Discount         float64
Profit           float64
dtype: object

Missing Values:
Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID 

In [6]:
# Comprehensive EDA
print("Sales & Profit Analysis")
print(f"Sales Range: ${data['Sales'].min():,.2f} - ${data['Sales'].max():,.2f}")
print(f"Profit Range: ${data['Profit'].min():,.2f} - ${data['Profit'].max():,.2f}")

# Check for data corruption in numeric columns
print("\nData Quality Check")
for col in ['Sales', 'Profit', 'Quantity', 'Discount']:
    non_numeric = data[~pd.to_numeric(data[col], errors='coerce').notnull()]
    if len(non_numeric) > 0:
        print(f"Found {len(non_numeric)} corrupted rows in {col}:")
        print(non_numeric[[col, 'Product Name', 'Row ID']].head())
    else:
        print(f"{col} column is clean")

# Regional Performance
print("\nRegional Performance")
regional_stats = data.groupby('Region').agg({
    'Sales': ['sum', 'mean'],
    'Profit': ['sum', 'mean'],
    'Quantity': 'sum'
}).round(2)
print(regional_stats)

# Category Performance
print("\nCategory Performance")
category_stats = data.groupby('Category').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum'
}).round(2)
category_stats['Profit_Margin_Pct'] = (category_stats['Profit'] / category_stats['Sales'] * 100).round(2)
print(category_stats)

Sales & Profit Analysis
Sales Range: $0.44 - $22,638.48
Profit Range: $-6,599.98 - $8,399.98

Data Quality Check
Sales column is clean
Profit column is clean
Quantity column is clean
Discount column is clean

Regional Performance
             Sales             Profit        Quantity
               sum    mean        sum   mean      sum
Region                                               
Central  501239.89  215.77   39706.36  17.09     8780
East     678781.24  238.34   91522.78  32.14    10618
South    391721.90  241.80   46749.43  28.86     6209
West     725457.82  226.49  108418.45  33.85    12266

Category Performance
                     Sales     Profit  Quantity  Profit_Margin_Pct
Category                                                          
Furniture        741999.80   18451.27      8028               2.49
Office Supplies  719047.03  122490.80     22906              17.04
Technology       836154.03  145454.95      6939              17.40


In [7]:
# Time series analysis
data['Order Date'] = pd.to_datetime(data['Order Date'])
data['Year_Month'] = data['Order Date'].dt.to_period('M')

print("Time Series Trends")
monthly_trends = data.groupby('Year_Month').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Order ID': 'nunique'
}).tail(12)
print(monthly_trends)

# Top/bottom performers
print("\nTop 10 Profitable Products")
top_products = data.groupby('Product Name')['Profit'].sum().nlargest(10)
print(top_products)

print("\nBottom 10 Loss-Making Products")
loss_products = data.groupby('Product Name')['Profit'].sum().nsmallest(10)
print(loss_products)

# Customer segments
print("\nCustomer Segment Analysis")
segment_stats = data.groupby('Segment').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Customer ID': 'nunique',
    'Order ID': 'nunique'
}).round(2)
segment_stats['Avg_Order_Value'] = (segment_stats['Sales'] / segment_stats['Order ID']).round(2)
print(segment_stats)

# Shipping mode analysis
print("\nShipping Mode Analysis")
ship_stats = data.groupby('Ship Mode').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Order ID': 'nunique'
}).round(2)
print(ship_stats)

Time Series Trends
                  Sales      Profit  Order ID
Year_Month                                   
2017-01      43971.3740   7140.4391        69
2017-02      20301.1334   1613.8720        53
2017-03      58872.3528  14751.8915       118
2017-04      36521.5361    933.2900       116
2017-05      44261.1102   6342.5828       118
2017-06      52981.7257   8223.3357       133
2017-07      45264.4160   6952.6212       111
2017-08      63120.8880   9040.9557       111
2017-09      87866.6520  10991.5556       226
2017-10      77776.9232   9275.2755       147
2017-11     118447.8250   9690.1037       261
2017-12      83829.3188   8483.3468       224

Top 10 Profitable Products
Product Name
Canon imageCLASS 2200 Advanced Copier                                          25199.9280
Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind     7753.0390
Hewlett Packard LaserJet 3310 Copier                                            6983.8836
Canon PC1060 Personal Lase

In [9]:
from google.colab import files
# Copy data and ensure numeric columns
clean_data = data.copy()
numeric_cols = ['Sales', 'Profit', 'Quantity', 'Discount']
initial_rows = len(clean_data)

for col in numeric_cols:
    clean_data[col] = pd.to_numeric(clean_data[col], errors='coerce')

clean_data.dropna(subset=numeric_cols, inplace=True)
print(f"Removed {initial_rows - len(clean_data)} corrupted rows")

# Convert dates once
order_dt = pd.to_datetime(clean_data['Order Date'])
ship_dt = pd.to_datetime(clean_data['Ship Date'])

# Calculated features
clean_data['order_year'] = order_dt.dt.year
clean_data['order_month'] = order_dt.dt.month
clean_data['order_quarter'] = order_dt.dt.quarter
clean_data['days_to_ship'] = (ship_dt - order_dt).dt.days
clean_data['profit_margin'] = (clean_data['Profit'] / clean_data['Sales']) * 100
clean_data['revenue_per_unit'] = clean_data['Sales'] / clean_data['Quantity']
clean_data['profit_per_unit'] = clean_data['Profit'] / clean_data['Quantity']

# Handle infinite/NaN
clean_data.replace([np.inf, -np.inf], 0, inplace=True)
clean_data.fillna(0, inplace=True)

# Profit categories
clean_data['profit_category'] = pd.cut(
    clean_data['Profit'],
    bins=[-np.inf, 0, 50, 200, np.inf],
    labels=['Loss', 'Low_Profit', 'Medium_Profit', 'High_Profit']
)

# MySQL-compatible dates
clean_data['order_date'] = order_dt.dt.strftime('%Y-%m-%d')
clean_data['ship_date'] = ship_dt.dt.strftime('%Y-%m-%d')

# Final clean dataframe
final_cols = {
    'Row ID':'row_id', 'Order ID':'order_id', 'Ship Mode':'ship_mode', 'Customer ID':'customer_id',
    'Customer Name':'customer_name', 'Segment':'segment', 'Country':'country', 'City':'city',
    'State':'state', 'Postal Code':'postal_code', 'Region':'region', 'Product ID':'product_id',
    'Category':'category', 'Sub-Category':'sub_category', 'Product Name':'product_name',
    'Sales':'sales', 'Quantity':'quantity', 'Discount':'discount', 'Profit':'profit'
}

final_data = clean_data[list(final_cols.keys())].rename(columns=final_cols)
final_data['postal_code'] = pd.to_numeric(final_data['postal_code'], errors='coerce').fillna(0).astype(int)
final_data['sales'] = clean_data['Sales'].round(4)
final_data['quantity'] = clean_data['Quantity'].astype(int)
final_data['discount'] = clean_data['Discount'].round(4)
final_data['profit'] = clean_data['Profit'].round(4)

# Add calculated features
for col in ['order_year','order_month','order_quarter','days_to_ship',
            'profit_margin','revenue_per_unit','profit_per_unit','profit_category','order_date','ship_date']:
    final_data[col] = clean_data[col]

# Clean text fields
text_fields = [
    'order_id','ship_mode','customer_id','customer_name','segment','country','city','state',
    'region','product_id','category','sub_category','product_name','profit_category'
]
final_data[text_fields] = final_data[text_fields].replace({'"':'', "'":'', ',':''}, regex=True)

# Export CSV
final_data.to_csv('SUPERSTORE_FINAL.csv', index=False, quoting=0)
files.download('SUPERSTORE_FINAL.csv')

Removed 0 corrupted rows


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>